### 1. Połączenie z Digital Ocean Spaces

In [ ]:
# potrzebne biblioteki do pracy z Cloud Storage
from dotenv import load_dotenv
import boto3
import os
import pandas as pd
from pycaret.datasets import get_data 

# wczytanie zmiennych środowiskowych
load_dotenv()

# nawiązanie połączenia z Digital Ocean Spaces
session = boto3.session.Session()
client = session.client(
    "s3",
    #aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    #aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    #endpoint_url=os.getenv("AWS_ENDPOINT_URL_S3")
)

In [ ]:
BUCKET_NAME = "the-most-important-variables"

### 2. Wygenerowanie datasetów

In [ ]:
dataset_df = get_data('index', verbose=False)
# filtruję tylko te datasety związane z regresją lub klasyfikacją
filtered_df = dataset_df[dataset_df['Default Task'].isin(['Classification (Binary)',
'Regression', 'Classification (Multiclass)'])]
# tworzę listę nazw wyfiltrowanych datasetów
datasets_list = filtered_df['Dataset'].tolist()
#datasets_list

In [19]:
datasets_list

['bank',
 'blood',
 'cancer',
 'credit',
 'diabetes',
 'electrical_grid',
 'employee',
 'heart',
 'heart_disease',
 'hepatitis',
 'income',
 'juice',
 'nba',
 'wine',
 'telescope',
 'titanic',
 'us_presidential_election_results',
 'glass',
 'iris',
 'poker',
 'questions',
 'satellite',
 'CTG',
 'automobile',
 'bike',
 'boston',
 'concrete',
 'diamond',
 'energy',
 'forest',
 'gold',
 'house',
 'insurance',
 'parkinsons',
 'traffic']

### 3. Wysłanie wyfiltrowanych datasetów do Digital Ocean Spaces

In [ ]:
folder_name = 'datasets/' 

for dataset_name in datasets_list:
    
    ready_df = get_data(dataset_name)
    
    if not ready_df.empty:
        csv_data = ready_df.to_csv(index = False)

        client.put_object(Bucket=BUCKET_NAME, Key=f'{folder_name}{dataset_name}.csv', Body=csv_data)
        print(f'Uploaded {dataset_name}.csv to Digital Ocean Spaces')        

### Wczytanie datasetów z Digital Ocean Spaces do DataFrame

In [ ]:
# wczytuję przykładowy dataset o dokładnej nazwie
df = pd.read_csv(f's3://{BUCKET_NAME}/{folder_name}titanic.csv', sep=',')
df.head()